# Cleveland Heart Disease Classification with Neural Networks

## 1. Import libraries, datasets and observe the data

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# in Google Colab:
# df = pd.read_csv('/content/drive/MyDrive/Datasets/heart.csv')

# in Kaggle
df = pd.read_csv('/kaggle/input/cleveland-heat-disease/heart.csv')

In [ ]:
df.head()

In [ ]:
# Shape of the data
df.shape

The data has 1025 samples (rows) and 14 features (columns). The features are following:

1. `age` - person's age in years
2. `sex` - the person's sex (1 = male, 0 = female)
3. `cp` chest main type - the chest pain experienced. Value 1: typical angina; Value 2: atypical angina; Value 3: non-anginal pain; Value 4: asymptomatic
4. `trestbps` Resting blood pressure - the person's resting blood pressure
5. `chol` cholesterol - the person's cholesterol measurement in mg/dl
6. `fbs` Fasting blood sugar - the person's fasting blood sugar (>120 mg/dl, 1 = true, 0 = false)
7. `restecg` resting ecg - resting electrocardiographic measurement.
8. `thalach`- max heart rate achieved
9. `exang`- exercise induced angina (1 = yes, 0 = no)
10. `oldpeak`-
11. `slope`
12. `ca` - number of vessels
13. `thal` - thalasemia
14. `target` - diagnosis of heart disease

In [ ]:
df.describe().T

In [ ]:
df.info()

All feastures are numeric columns.

## 2. Check and handle missing values

In [ ]:
pd.DataFrame(df.isnull().sum(), columns = ['Missing']).T

There is not any missing values in all columns.

## 3. Distribution and outliers

In [ ]:
import math

In [ ]:
cols = df.columns.to_list()
ncols = 3
nrows = int(np.ceil(len(cols) / ncols))

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 10))
axes = axes.flatten()

sns.set_theme(style="whitegrid", palette="pastel", font_scale=1.1)

for i, col in enumerate(cols):
    sns.boxplot(data=df, x=col, ax=axes[i], color=sns.color_palette("pastel")[1])
    axes[i].set_xlabel(col, fontsize=10, weight='bold')
    axes[i].set_ylabel("")
    axes[i].tick_params(axis='x', labelrotation=30)  # Rotate labels to avoid overlap
    axes[i].set_title("")  # Keep the plot clean, rely on x-labels

# Remove unused axes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Boxplots for Each Feature", fontsize=16, weight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
numeric_cols = ['trestbps', 'chol']

In [ ]:
def outlier_detection(df, col):
    q1 = np.percentile(df, 25)
    q3 = np.percentile(df, 75)
    iqr = q3 - q1

    lower, upper = q1-1.5*iqr, q3 + 1.5*iqr

    return lower, upper

In [ ]:
for col in numeric_cols:
    lower, upper = outlier_detection(df, col)

    df[col] = np.clip(df[col], a_min = lower, a_max = upper)

### Distribution

In [ ]:
cols = df.columns.to_list()
ncols = 3
nrows = int(np.ceil(len(cols) / ncols))

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, nrows * 3))
axes = axes.flatten()

sns.set_theme(style="whitegrid", palette="pastel", font_scale=1.1)

for i, col in enumerate(cols):
    sns.histplot(data=df, x=col, ax=axes[i], color=sns.color_palette("pastel")[1])
    axes[i].set_xlabel(col, fontsize=10, weight='bold')
    axes[i].set_ylabel("")
    axes[i].tick_params(axis='x', labelrotation=30)  # Rotate labels to avoid overlap
    axes[i].set_title("")  # Keep the plot clean, rely on x-labels

# Remove unused axes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Boxplots for Each Feature", fontsize=16, weight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
df.head()

## 4. Scaling of numeric variables

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X = df.drop('target', axis = 1).values
y = df['target'].values.reshape(-1,1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 6. Build model

In [ ]:
# Initialization

np.random.seed(42)
n_x = X_train.shape[1]   # should be 13
n_h1 = 6
n_h2 = 4
n_y = 1

Explanation of the code:

1. `X_train.shape[1]` - extract the number of the features and set it to input size. Because, each neuron in the input layer corresponds to one feature.
2. `n_h1 = 6` and `n_h2 = 4` - defines the number of neurons in the 1st and second hidden layers, respectively. these are design choices known as hyperparameters. They’re not fixed by theory but rather chosen based on experimentation, balancing model capacity and overfitting risk.
3. `n_y` - defines the output layer with a single neuron; because it is binary classification task (result can be either 0 or 1).


In [ ]:
X_train.shape

In [ ]:
# Weight initialization (small random numbers)
W1 = np.random.randn(n_x, n_h1) * 0.1
b1 = np.zeros((1, n_h1))

W2 = np.random.randn(n_h1, n_h2) * 0.1
b2 = np.zeros((1, n_h2))

W3 = np.random.randn(n_h2, n_y) * 0.1
b3 = np.zeros((1, n_y))

The codes above initialize the weights and biases - which are the adjustable parameters that the network will learn during training. For each layer i, the matrix W (weights) connects layer i−1 to layer i, and b (biases) shifts the activation before it’s passed to the next layer.
-  `W1 = np.random.randn(n_x, n_h1) * 0.1` creates a 13×6 weight matrix drawn from a standard normal distribution (randn gives mean 0, variance 1) and scaled by 0.1 to make the initial values small—this prevents early layer activations from exploding.
- The bias `b1 = np.zeros((1, n_h1))` initializes as zeros, giving every neuron in that layer the same initial offset (biases can safely start at zero because random weights already break symmetry)

The same logic applies to W2 and b2 (6×4 matrix) for the second hidden layer, and W3 and b3 (4×1 matrix) for the output layer.

The random initialization gives each neuron a unique starting point so they learn distinct features rather than identical patterns. The scaling factor (0.1) ensures the network starts in a numerically stable regime, allowing gradients to flow effectively during backpropagation.


In [ ]:
# Sigmoid activation function

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

The sigmoid function is one of the most commonly used activation functions in neural networks, especially in binary classification problems. It takes any real-valued input and squashes it into a range between 0 and 1, which makes it useful for representing probabilities.When the input is a large positive number, the output gets close to 1, and when it is a large negative number, the output approaches 0. However, one limitation of the sigmoid function is that it can cause the <font color = red>vanishing gradient problem </font> during training, which slows down learning for deep networks.

 **Vanishing gradient problem** - happens when the gradients (which tell the network how much to adjust the weights during training) become extremely small as they move backward through the layers of a neural network.

 In the case of the sigmoid function, when the input value is very large or very small, the slope (derivative) of the sigmoid becomes almost zero because the output is close to 1 or 0 and the curve flattens out. During backpropagation, these tiny gradients are multiplied layer by layer, and as a result, they get smaller and smaller. This means the earlier layers of the network receive almost no updates — they “learn” extremely slowly or sometimes not at all.

 In simpler terms, the vanishing gradient problem makes it difficult for deep neural networks using sigmoid activation to learn effectively, especially in the deeper layers. That’s one reason why activation functions like ReLU (Rectified Linear Unit) are preferred in modern deep learning — they help keep gradients from vanishing.

In [ ]:
def relu(z):
    return np.maximum(0, z)

In [ ]:
def relu_deriv(z):
    return (z > 0).astype(float)

The binary cross-entropy loss (also called log loss) is one of the most common loss functions used for binary classification problems — where the output can be either 0 or 1. Its formula is written as:

$$
L = -\frac{1}{m} \sum_{i=1}^{m} \Big[ y_i \log(\hat{y}_i + \varepsilon) + (1 - y_i)\log(1 - \hat{y}_i + \varepsilon) \Big]
$$

where

- m - total number of samples
- $y_i$ - true label of the ith sample (0 or 1)
- $\hat{y_i}$ - predicted probability for that sample
- $\epsilon$ - a very small constant

If the model predicts correctly with high confidence (e.g., $\hat{y_i}=0.99$ when $\hat{y_i}=1$ ) the logarithm term becomes small, meaning a low loss. f it predicts incorrectly, the log term explodes to a large value, meaning a high loss. This loss function encourages the model to assign high probabilities to correct classes while penalizing overconfident wrong predictions.

In [ ]:
# Loss: binary cross-entropy
def compute_loss(y_true, y_pred):
    m = y_true.shape[0]
    eps = 1e-8
    return - (1.0 / m) * np.sum(y_true * np.log(y_pred + eps) + (1 - y_true) * np.log(1 - y_pred + eps))


The next code chunk is **forward pass** function.

The forward pass is the process where input data flows through the neural network to generate predictions. It starts by computing the first linear transformation `z1 = X.dot(W1) + b1`, where the input features `X` are multiplied by the first layer’s weights `W1` and added to the bias `b1`. This produces the raw input to the first hidden layer, which then passes through the ReLU activation function `a1 = relu(z1)` to introduce nonlinearity.

The same steps repeat for the second hidden layer: `z2 = a1.dot(W2) + b2` computes the linear combination, and `a2 = relu(z2)` applies the activation.

Finally, the output layer computes `z3 = a2.dot(W3) + b3`, followed by a sigmoid activation `a3 = sigmoid(z3) to produce values between 0 and 1, representing the predicted probabilities for the target class.

In [ ]:
# Forward pass (returns intermediates needed for backprop)
def forward(X):
    z1 = X.dot(W1) + b1          # (m, n_h1)
    a1 = relu(z1)
    z2 = a1.dot(W2) + b2         # (m, n_h2)
    a2 = relu(z2)
    z3 = a2.dot(W3) + b3         # (m, 1)
    a3 = sigmoid(z3)
    cache = (z1, a1, z2, a2, z3, a3)
    return a3, cache

# The function also stores all intermediate results—z1, a1, z2, a2, z3, and
# a3—in a variable called cache, which is later used during backpropagation to
# compute gradients.

The next process is backpropagation algorithm.

Backpropagation is the method by which the network computes gradients of the loss with respect to each weight and bias, allowing the parameters to be adjusted in the direction that reduces the loss.

The function starts by unpacking the cached intermediate results from the forward pass `(z1, a1, z2, a2, z3, a3)` and calculating the number of samples `m`.

1. **Output layer gradients**:
 - `dz3 = a3 - y` just measures the difference between the predicted probability and the true label — basically “how wrong the output is.”
 - `dW3` and `db3` tell us how much the weights and biases in the output layer contributed to that error. We use them to update the weights so the output gets closer to the true label.

 2. **Second hidden layer**:
 - The error from the output layer is sent backward: `da2 = dz3.dot(W3.T)` shows how much each neuron in the second hidden layer caused the output error.
 - Multiplying by `relu_deriv(z2)` adjusts for the ReLU activation, because ReLU only passes gradients for positive values.
 - `dW2` and `db2` then tell us how much to change the weights and biases in this layer.

 3. **First hidden layer**:
 - Same idea: `da1 = dz2.dot(W2.T)` passes the error back one more layer.
 - Multiply by `relu_deriv(z1)` to adjust for ReLU.
 - `dW1` and `db1` give the updates for the first hidden layer weights and biases.


---
**Key Points:**
- Start at the output: see the error.

- Pass the error backward, layer by layer, adjusting for the activation function.

- Compute how much each weight/bias contributed to the error.

- Update the weights/biases to reduce the error.

In one sentence: backpropagation is just figuring out “who caused the error” in the network and nudging those weights in the right direction.

After buildin the network structure, the next part is making predictions on the train data and then train the data based on the results.


In [ ]:
# Backprop and parameters update (full-batch gradient descent)
def backward_and_update(X, y, cache, lr=0.01):
    global W1, b1, W2, b2, W3, b3
    m = X.shape[0]
    z1, a1, z2, a2, z3, a3 = cache

    # Output layer gradients
    dz3 = a3 - y                       # (m,1)
    dW3 = (a2.T.dot(dz3)) / m          # (n_h2, 1)
    db3 = np.sum(dz3, axis=0, keepdims=True) / m

    # Layer 2
    da2 = dz3.dot(W3.T)                # (m, n_h2)
    dz2 = da2 * relu_deriv(z2)         # (m, n_h2)
    dW2 = (a1.T.dot(dz2)) / m          # (n_h1, n_h2)
    db2 = np.sum(dz2, axis=0, keepdims=True) / m

    # Layer 1
    da1 = dz2.dot(W2.T)                # (m, n_h1)
    dz1 = da1 * relu_deriv(z1)         # (m, n_h1)
    dW1 = (X.T.dot(dz1)) / m           # (n_x, n_h1)
    db1 = np.sum(dz1, axis=0, keepdims=True) / m

    # Update params
    W3 -= lr * dW3
    b3 -= lr * db3
    W2 -= lr * dW2
    b2 -= lr * db2
    W1 -= lr * dW1
    b1 -= lr * db1


For the prediction part we will have two functions:

- `predict_proba(X)` : runs a forward pass through the network using the input X and returns the output layer activations a3. Since the output layer uses a sigmoid, these values are probabilities between 0 and 1, representing the model’s confidence that each sample belongs to class 1.

- `predict(X, threshold=0.5)`: uses `predict_proba(X)` to get the probabilities, then converts them into class labels. It compares each probability to the threshold (default 0.5): if the probability is greater than or equal to 0.5, the function assigns it to class 1; otherwise, it assigns class 0. The .astype(int) ensures the output is an integer array of 0s and 1s.

In short, predict_proba will give the probabilities, while predict will return binary class predictions, which are useful for computing accuracy or making final decisions.

In [ ]:
# Helpers: predict probabilities and classes
def predict_proba(X):
    a3, _ = forward(X)
    return a3

def predict(X, threshold=0.5):
    probs = predict_proba(X)
    return (probs >= threshold).astype(int)


Now it is time to write code for training.


In [ ]:
# Training loop
# -------------------------
epochs = 1500
lr = 0.05   # you can reduce if loss diverges
print_every = 100

for epoch in range(1, epochs + 1):
    # forward
    y_hat_train, cache = forward(X_train)
    loss = compute_loss(y_train, y_hat_train)

    # backward + update
    backward_and_update(X_train, y_train, cache, lr=lr)

    # logging
    if epoch % print_every == 0 or epoch == 1:
        train_preds = predict(X_train)
        test_preds  = predict(X_test)

        train_acc = np.mean(train_preds == y_train)
        test_acc  = np.mean(test_preds == y_test)

        print(f"Epoch {epoch:4d} | loss: {loss:.4f} | train_acc: {train_acc:.4f} | test_acc: {test_acc:.4f}")


The above code consists of the following parts:
1.  <font color = darkcyan><b>Set hyperparameters:</b></font>
- `epochs` sets how many times the network will see the entire training dataset. More epochs allow more learning but take longer.
- `lr (learning rate)` controls the size of the steps taken during weight updates. Too large can make the loss explode; too small makes learning slow.
- `print_every` determines how often we print progress to monitor training.

2. <font color = darkcyan><b>Loop over epochs:</b></font>

- We iterate from 1 to epochs so that each pass through the loop is one full training cycle over the dataset.

3.  <font color = darkcyan><b>Forward pass:</b></font>
- `forward(X_train)` computes the network’s predictions for all training samples.
- `y_hat_train` contains the predicted probabilities for each sample.
- `cache` stores intermediate values needed for backpropagation.
- `compute_loss` calculates the binary cross-entropy loss, measuring how far predictions are from the true labels.

4.  <font color = darkcyan><b>Backward pass and update:</b></font>
- This function computes the gradients of the loss with respect to all weights and biases (backpropagation) and updates them using gradient descent with learning rate lr. This is the step where the network “learns.”

5.  <font color = darkcyan><b>Logging / monitoring:</b></font>

- Every print_every epochs (and the first epoch), we check how well the network is performing:
    - `predict(X_train)` converts probabilities into class labels for the training set.
    - `predict(X_test)` does the same for the test set.

6.  <font color = darkcyan><b>Accuracy calculation:</b></font>
- We compute the fraction of correctly predicted samples for training and test sets. This is a simple measure of model performance.

7.  <font color = darkcyan><b>Printing progress:</b></font>

- This line prints a summary showing:
    - The current epoch

    - Training loss

    - Training accuracy

    - Test accuracy

    It helps track if the network is learning over time, if loss is decreasing, and whether the model is overfitting (training accuracy high but test accuracy low).

In [ ]:
# Final evaluation
train_probs = predict_proba(X_train)
test_probs  = predict_proba(X_test)

train_acc = np.mean((train_probs >= 0.5).astype(int) == y_train)
test_acc  = np.mean((test_probs  >= 0.5).astype(int) == y_test)
print("\nFinal -> Train accuracy: {:.4f} | Test accuracy: {:.4f}".format(train_acc, test_acc))


Accuracy alone is often not enough for evaluating classification models, especially when the classes are imbalanced, like in many medical datasets (including Cleveland heart disease). Accuracy just measures the fraction of correct predictions, but it doesn’t tell you how well the model handles false positives and false negatives, which is where metrics like precision, recall, F1-score, ROC AUC, and Gini coefficient come in.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Convert probabilities to class predictions
train_preds = (train_probs >= 0.5).astype(int)
test_preds  = (test_probs >= 0.5).astype(int)

# Precision, Recall, F1
train_precision = precision_score(y_train, train_preds)
train_recall    = recall_score(y_train, train_preds)
train_f1        = f1_score(y_train, train_preds)

test_precision  = precision_score(y_test, test_preds)
test_recall     = recall_score(y_test, test_preds)
test_f1         = f1_score(y_test, test_preds)

# ROC AUC and Gini
train_auc = roc_auc_score(y_train, train_probs)
test_auc  = roc_auc_score(y_test, test_probs)

train_gini = 2 * train_auc - 1
test_gini  = 2 * test_auc - 1




# Create a dictionary with metrics
metrics_dict = {
    "Train": [train_acc,train_precision, train_recall, train_f1, train_auc, train_gini],
    "Test":  [test_acc,test_precision, test_recall, test_f1, test_auc, test_gini]
}

# Index for the DataFrame
metrics_index = ["Accuracy", "Precision", "Recall", "F1-score", "ROC AUC", "Gini"]

# Create DataFrame
metrics_df = pd.DataFrame(metrics_dict, index=metrics_index)


In [ ]:
metrics_df

Looking at these results, we can summarize the neural network’s performance on the Cleveland heart disease dataset as follows:

<font color = skyblue><b>Accuracy:</b></font> The neural network correctly predicts the class for about 89% of the training samples and 83% of the test samples. This indicates good overall performance, though the slightly higher training accuracy suggests minor overfitting. Accuracy alone is useful, but in medical datasets, other metrics like recall and precision are more critical.

<font color = skyblue><b>Precision:</b></font> The model is slightly better at avoiding false positives on the training set (0.87) than on the test set (0.80). This means when it predicts a patient has heart disease, it is more often correct on the training data.

<font color = skyblue><b>Recall:</b></font>  The model detects most of the actual positive cases, with very high recall on both training (0.93) and test (0.88). This is good for medical applications where missing positive cases is risky.

<font color = skyblue><b>F1-score:</b></font>  Balancing precision and recall, the F1-score is strong on both training (0.90) and test (0.84), showing overall good predictive power.

<font color = skyblue><b>ROC AUC:</b></font>  High AUC values (0.95 train, 0.90 test) indicate the model can distinguish between patients with and without heart disease effectively.

<font color = skyblue><b>Gini coefficient:</b></font>  Also high (0.90 train, 0.80 test), reinforcing strong discriminative ability.

<font color = steelred><b>Summary insight:</b></font>  The model performs very well overall, with slightly better results on the training set, which is expected. There is a small gap between train and test metrics, suggesting minor overfitting, but the test performance remains strong, especially recall and AUC, which are critical in a medical context to catch as many true cases as possible.